In [27]:
import json
import os
import math
import numpy as np
import pandas as pd
from scipy.special import comb

json_file = open("sp500_bb.json", 'r', encoding='utf-8')
desktop_path = os.path.expanduser("~/Desktop")
json_file_path = os.path.join(desktop_path, "sp500_bb.json")
# period_data = json.load("/Users/shuxianglei/Desktop/sp500_bb.json")
with open(json_file_path, "r") as f:
    period_data = json.load(f)
Bull = period_data['bull']
Bear = period_data['bear']

print('Mean of Bull: ', np.mean(Bull), 'Mean of Bear: ', np.mean(Bear))
print("Stationary Prob of Bull:", round(np.mean(Bull) / (np.mean(Bull) + np.mean(Bear)) * 100, 2),
      "Stationary Prob of Bear:", round(np.mean(Bear) / (np.mean(Bull) + np.mean(Bear)) * 100, 2))

for q in range(1, 7):  # NB(1)  NB(2)  NB(3)  NB(4)  NB(5)  NB(6)
    p_bull = len(Bull) * q / (len(Bull) * q + sum(Bull))  # transition probability between two sub-states
    p_bear = len(Bear) * q / (len(Bear) * q + sum(Bear))  # transition probability between two sub-states
    L_bull_l = [np.log(max(1, comb(int(Bull[i]) - 1, int(Bull[i]) - q))) + q * np.log(p_bull) + (Bull[i] - q) * np.log(
        1 - p_bull) for i in range(len(Bull))]
    L_bull = np.sum(L_bull_l)
    L_bear_l = [np.log(max(1, comb(int(Bear[i]), q - 1))) + q * np.log(p_bear) + (Bull[i] - q) * np.log(1 - p_bear) for
                i in range(len(Bear))]
    L_bear = np.sum(L_bear_l)
    print(q, round(p_bull, 2), round(L_bull, 2), round(p_bear, 2), round(L_bear, 2))
    P = np.zeros((2 * q, 2 * q))
    for i in range(q):
        P[i, i] = 1 - p_bull
        P[i, i + 1] = p_bull
        P[i + q, i + q] = 1 - p_bear
        if i + q + 1 < q * 2:
            P[i + q, i + q + 1] = p_bear
        else:
            P[i + q, 0] = p_bear
    # print(P)

    Pk_list = [P]
    Pk = P
    PAAk = [np.sum(Pk[:q,:q])/q]
    PBBk = [np.sum(Pk[q:,q:])/q]
    for k in range(300):
        Pk = Pk @ P
        if k < 29:
            Pk_list.append(Pk)
            PAAk.append(np.sum(Pk[:q,:q])/q)
            PBBk.append(np.sum(Pk[q:,q:])/q)
        elif k == 299:
            # print(Pk)
            pi_bull = Pk[0, 0] / (Pk[0, 0] + Pk[0, q])
            pi_bear = Pk[0, q] / (Pk[0, 0] + Pk[0, q])
print(len(PAAk))
print(len(PBBk))

miu = pi_bull*np.mean(Bull) + pi_bear*np.mean(Bear)
var = pi_bull * 0.007626591368227733 + pi_bear * 0.011494295898437498 + pi_bear*pi_bull*((np.mean(Bull)-np.mean(Bear))**2)
expection = []
for i in range(0, len(PAAk)):
    expection.append(pi_bull*np.mean(Bull)*(PAAk[i]*np.mean(Bull) + (1 - PAAk[i])*np.mean(Bear)) + pi_bear*np.mean(Bear)*((1-PBBk[i])*np.mean(Bull) + PBBk[i]*np.mean(Bear)))
# print(expection)

rhok = []
for i in range(0, len(expection)):
    rhok.append((expection[i] - miu**2) / var)
# print(rhok)

Rpp = np.zeros((30,30))
for i in range(30):
    for j in range(30):
        Rpp[i,j] = rhok[abs(i-j)-1] if i!=j else 1
# print(Rpp)

# Then compute AR coefficient Phi_p, make the sum equal to 1
Phi_p = np.linalg.inv(Rpp)@np.array(rhok)
theta = Phi_p / np.sum(Phi_p)
print(theta)

Rnp = np.zeros((30,30))
for i in range(30):
    for j in range(30):
        Rpp[i,j] = rhok[abs(i-j)-1] if i!=j else 1

theta1 = theta.T

ONE = np.ones((30,1))

m = theta1*ONE*miu

Rnn = Rpp.copy()

v = np.sqrt(theta1*Rnn*theta*var)

d = -(m/v)



Mean of Bull:  24.162636363636363 Mean of Bear:  11.016625
Stationary Prob of Bull: 68.68 Stationary Prob of Bear: 31.32
1 0.04 -137.43 0.08 -145.53
2 0.08 -142.34 0.15 -176.81
3 0.11 -152.49 0.21 -214.48
4 0.14 -166.17 0.27 -252.2
5 0.17 -181.61 0.31 -289.62
6 0.2 -198.75 0.35 -321.22
30
30
[ 1.2098334  -0.0034279  -0.00448771 -0.00407641 -0.00369766 -0.00468172
 -0.00703903 -0.00994867 -0.01252034 -0.01418723 -0.01477088 -0.0143834
 -0.01329138 -0.0118027  -0.01019377 -0.00867369 -0.00737469 -0.00635842
 -0.0056304  -0.00515701 -0.00488184 -0.00473924 -0.00466423 -0.00459876
 -0.00449534 -0.00432018 -0.00405767 -0.00371455 -0.00331495 -0.00934362]


/Users/shuxianglei/opt/anaconda3/envs/practicum/lib/python3.7/site-packages/ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in sqrt
